In [4]:
import tensorflow.keras as keras
import json


import sys
import tensorflow as tf
#converts jupyter to txt file, then you change it to .py
!jupyter nbconvert --to script score_preprocessing.ipynb
sys.path.append(".")
#create a folder called dataset_mel BEFORE RUNNING!!!!!!!
from score_preprocessing import seq_length

[NbConvertApp] Converting notebook score_preprocessing.ipynb to script
[NbConvertApp] Writing 5723 bytes to score_preprocessing.txt
12
good duration - True
loadin
loaded 12 songs
key changed
key changed
key changed
key changed


/usr/local/lib/python3.10/dist-packages/music21/stream/base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


key changed
key changed
key changed
key changed
key changed
key changed
key changed
key changed
55 _ _ _ 64 _ _ _ 64 _ 64 _ 64 _ _ _ 62 _ 62 _ 62 _ _ _ _ _ 60 _ 60 _ _ _ 55 _ _ _ 65 _ _ _ 65 _ 65 _ 65 _ 64 _ 67 _ 64 _ 64 _ _ _ 62 _ _ _ rest _ _ _ 55 _ _ _ 67 _ _ _ 67 _ 67 _ 67 _ _ _ 65 _ 65 _ 64 _ _ _ _ _ _ _ 67 _ 65 _ 64 _ 62 _ 60 _ _ _ 60 _ 60 _ 64 _ 62 _ 62 _ _ _ 60 _ _ _ _ _ _ _ 62 _ _ _ 65 _ _ _ 64 _ _ _ 64 _ 64 _ 67 _ 65 _ 64 _ 62 _ 60 _ _ _ _ _ _ _ rest _ _ _ / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / 64 _ 69 _ _ _ 71 _ 72 _ _ 71 69 _ 76 _ _ _ _ _ 71 _ _ _ 74 _ 72 _ _ 71 69 _ 68 _ _ _ 69 _ 71 _ _ _ _ _ rest _ _ _ 64 _ 76 _ _ _ 76 _ 72 _ _ 71 69 _ 81 _ _ _ _ _ 77 _ _ _ 74 _ 72 _ _ 71 69 _ 71 _ _ 69 68 _ 69 _ _ _ _ _ rest _ _ _ / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / 76 _ _ _ _ _ 74 _ 72 _ 64 _ 69 _ _ _ 67 _ 65

In [5]:
import numpy as np
import music21 as m21

In [6]:
print(seq_length)
tab_path = 'tab_dict.json'

64


In [29]:
class Generator:
  def __init__(self, model_path = "Lstm_melody.h5"):

    self.model_path = model_path
    self.model = keras.models.load_model(self.model_path)

    with open(tab_path, 'r') as fp:
      self.mappings = json.load(fp)

    self.slashes = ["/"] * seq_length #


  def sampleANDtemp(self, probabilites, temperature):
    # temp -> inf, prob become even, as if randomly picking
    # temp -> 0, highest prob then becomes 1 - more deterministic
    # temp -> 1, normal sampling from softmax
    predictions = np.log(probabilites) / temperature
    probabilites = np.exp(predictions) / np.sum(np.exp(predictions))

    choices = range(len(probabilites))
    index = np.random.choice(choices, p = probabilites)
    #assign probabilties to the index to pick a next note

    return index

  def generate_mel(self, melo, num_steps,  max_sequence_length, temp):
    #melo is encoded melody
    #num steps is how many steps we want to generate
    melo = melo.split()

    melody = melo
    melo = self.slashes + melo #add slashes before start

    #map to tabdict values
    melo = [self.mappings[note] for note in melo]

    for i in range(num_steps):
      #use max_sequence_length to generate
      melo = melo[-max_sequence_length:] #64 from end to end

      encoded_melo = keras.utils.to_categorical(melo, num_classes = len(self.mappings))
      #turns into matrix of basis vectors
      encoded_melo = encoded_melo[np.newaxis, ...]

      #predicts
      probabilities = self.model.predict(encoded_melo)[0]
      #list of all output values,
      #softmax makes the outputs add to one, partion function from statistical physics.

      output_val = self.sampleANDtemp(probabilities, temp)

      melo.append(output_val)

      #map to int to key from tab_dict.
      #map from network output values to midi note representation

      output_note = [key for key, val in self.mappings.items() if val == output_val][0]
      if output_note == "/":
        break

      melody.append(output_note)

    return melody

  def save_gen(self, melody, step_duration=0.25, format="midi", file_name="mel.mid"):



        stream = m21.stream.Stream()#creates single part stream
        #defaults to 4/4 time which is fine. step dur =0.25 -> 16th note, since 1 is quarter note.

        start_val = None
        step_counter = 1

        # symbols and rests to midi
        for i, val in enumerate(melody):

            # handle case in which we have a note/rest
            if (val != "_") or ((i + 1) == len(melody)):

                # check note
                if start_val is not None:

                    quarter_length_duration = step_duration * step_counter
                    # length in terms of quarter notes

                    # handle rest
                    if start_val == "rest":
                        m21_event = m21.note.Rest(quarterLength=quarter_length_duration)

                    # handle note
                    else:
                        m21_event = m21.note.Note(int(start_val), quarterLength=quarter_length_duration)
                         #values are stored as strings so need to change to int

                    stream.append(m21_event)

                    # reset the step counter
                    step_counter = 1

                start_val = val

            #held note
            else:
                step_counter += 1

        #writes to midi file
        stream.write(format, file_name)

























In [33]:
generate = Generator()
melo = "72 _ _ _ _ _ 72 _ _ _ _ _ 72 _ 71"
melody = generate.generate_mel(melo, num_steps = 500, max_sequence_length = 64, temp = 3)
print(melody)

#seems to get stuck on the same note if temperature is low
#when temperature is set above 1 the results seem alot better

#train with less epochs maybe??
#or smaller network since not using a huge amount of data??

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
['72', '_

In [34]:
generate.save_gen(melody)